Notebook purpose:

- Explore feasibility of identifying number of children

Historical child benefit rates:

- https://revenuebenefits.org.uk/child-benefit/guidance/how-much-can-your-client-get/rates-and-tables/

In [5]:
import sys

import pandas as pd
import s3fs

sys.path.append("/Users/fgu/dev/projects/entropy")
import entropy.data.cleaners as cl
import entropy.data.creators as cr
import entropy.data.make_data as md
import entropy.data.selectors as sl
import entropy.helpers.aws as ha
import entropy.helpers.data as hd
import entropy.helpers.helpers as hh

pd.set_option("display.max_rows", 120)
pd.set_option("display.max_columns", 120)
pd.set_option("max_colwidth", None)
%load_ext autoreload
%autoreload 2

fs = s3fs.S3FileSystem(profile="3di")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [41]:
df = hd.read_sample("XX7")

Time for read_sample                   : 3.78 minutes


In [50]:
import collections

WeeklyBenefitsRates = collections.namedtuple(
    "WeeklyBenefitsRates", ("first", "subsequent")
)

rates = [
    WeeklyBenefitsRates(21.05, 13.95),
    WeeklyBenefitsRates(20.70, 13.70),
]

rates

[WeeklyBenefitsRates(first=21.05, subsequent=13.95),
 WeeklyBenefitsRates(first=20.7, subsequent=13.7)]

Approach 1
- Enumerate all possible amounts for up to 5 children




In [47]:
def child_benefits(df):

    return (
        df.loc[df.tag.eq("benefits") & df.desc.str.contains("chb")]
        .set_index("date")
        .loc["Apr 2019":"March 2020"]
        .sort_values(["user_id", "date"])
    )


def child_benefits(df):
    return df[df.amount.eq(-137.6)].groupby("user_id").size()


child_benefits(df)

user_id
37        28
4257      37
5967      11
11687      6
17717     12
          ..
581147    17
583677    37
583957    37
584537    10
588317    14
Length: 348, dtype: int64